# Classification par plus proches voisins

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import load_digits, load_iris

## Données

Quelques jeux de données à classifier.

### Disque

In [ ]:
def obtenir_disque(n_echantillons=100):
    '''Echantillons dans le carre unite; classe 1 si dans le disque unite.'''
    X = np.random.uniform(-1, 1, size=(n_echantillons, 2))
    y = (np.linalg.norm(X, axis=1) <= 1).astype(int)
    return X, y

In [ ]:
X, y = obtenir_disque()

In [ ]:
X.shape

In [ ]:
np.unique(y)

In [ ]:
def montrer_disque(X, y):
    plt.figure(figsize=(4,4))
    plt.xlim(-1.1, 1.1)
    plt.ylim(-1.1, 1.1)
    x = np.linspace(-1, 1, 100)
    plt.plot(x, np.sqrt(1 - x**2), color='k')
    plt.plot(x, -np.sqrt(1 - x**2), color='k')
    plt.scatter(X[y==1, 0], X[y==1, 1])
    plt.scatter(X[y==0, 0], X[y==0, 1])
    plt.xticks([])
    plt.yticks([])

In [ ]:
montrer_disque(X, y)

In [ ]:
def montrer_classes(y):
    n_classes = len(np.unique(y))
    plt.figure(figsize=(n_classes,2))
    plt.hist(y, bins=5*n_classes, color='k')
    plt.xlabel('Classe')
    plt.ylabel('Nombre')
    plt.xticks(np.unique(y))
    plt.show()

In [ ]:
montrer_classes(y)

### Iris

In [ ]:
iris = load_iris()

In [ ]:
# types d'iris
print(iris.target_names)

In [ ]:
# caracteristiques
print(iris.feature_names)

In [ ]:
def obtenir_iris():
    '''Jeu de donnees Iris.'''    
    X = iris.data
    y = iris.target
    return X, y

In [ ]:
X, y = obtenir_iris()

In [ ]:
X.shape

In [ ]:
np.unique(y)

In [ ]:
def montrer_iris(X, y, dimensions=[0,1]):
    plt.figure(figsize=(4,4))
    for label in np.unique(y):
        plt.scatter(X[y==label, dimensions[0]], X[y==label, dimensions[1]], label=iris.target_names[label])
    plt.xlabel(iris.feature_names[dimensions[0]])
    plt.ylabel(iris.feature_names[dimensions[1]])
    plt.legend()
    plt.show()

In [ ]:
montrer_iris(X, y, dimensions=[0,1])

In [ ]:
montrer_iris(X, y, dimensions=[2,3])

In [ ]:
montrer_classes(y)

### Chiffres

In [ ]:
digits = load_digits()

In [ ]:
def obtenir_chiffres():
    '''Jeu de donnees Digits.'''    
    X = digits.data
    y = digits.target
    return X, y

In [ ]:
X, y = obtenir_chiffres()

In [ ]:
X.shape

In [ ]:
np.unique(y)

In [ ]:
def montrer_chiffres(X, y, limit_max=10):
    '''Montre une sélection de chiffres.'''
    labels, nombres = np.unique(y, return_counts=True)
    nombre_max = min(np.max(nombres), limit_max)
    img = np.zeros((100, nombre_max*10))
    for i in range(10):
        index_label = np.where(y == i)[0][:limit_max]
        for j, echantillon in enumerate(index_label):
            img[i*10+1:i*10+9,j*10+1:j*10+9] = X[echantillon].reshape((8, 8))
    plt.imshow(img, cmap='binary')
    plt.xticks([])
    plt.yticks(5 + 10*np.arange(10), np.arange(10))

In [ ]:
def obtenir_echantillons(X, y, n_echantillons=10):
    '''Donne une sélection aléatoire d'échantillons pour chaque classe.'''
    index = []
    for label in np.unique(y):
        index_label = np.where(y == label)[0]
        replace = len(index_label) > n_echantillons
        index += list(np.random.choice(index_label, size=n_echantillons, replace=replace))
    return index

In [ ]:
index = obtenir_echantillons(X, y)
montrer_chiffres(X[index], y[index])

In [ ]:
montrer_classes(y)

## Partage apprentissage / test

Partage d'un jeu de données entre apprentissage et test.  

In [ ]:
def partager_apprentissage_test(X, y, ratio_test=0.2):
    '''Partage un jeu de données entre apprentissage et test. La repartition entre classes est conservee.'''
    index = []
    for label in np.unique(y):
        index_label = np.where(y==label)[0]
        index += list(np.random.choice(index_label, int(ratio_test * len(index_label)), replace=False))
    X_test = X[index]
    y_test = y[index]
    index_ = np.ones(len(y), dtype=bool)
    index_[index] = False
    X_app = X[index_]
    y_app = y[index_]
    return X_app, X_test, y_app, y_test

In [ ]:
X, y = obtenir_iris()

In [ ]:
X_app, X_test, y_app, y_test = partager_apprentissage_test(X, y)

In [ ]:
X_app.shape

In [ ]:
montrer_classes(y_app)

In [ ]:
X_test.shape

In [ ]:
montrer_classes(y_test)

## Plus proches voisins

In [ ]:
def rechercher_plus_proches_voisins(X_test, X_app, n_voisins=3):
    '''Recherche des plus proches voisins. Retourne une matrice de taille (n_test, n_voisins).'''
    voisins = []
    for x in X_test:
        distances = np.linalg.norm(X_app - x, axis=1)
        voisins.append(np.argpartition(distances, n_voisins)[:n_voisins])
    return np.array(voisins)

In [ ]:
def classifier_plus_proches_voisins(X_test, X_app, y_app, n_voisins=3):
    '''Classification par plus proches voisins. Retourne la classe majoritaire.'''
    y_pred = []
    for x in X_test:
        distances = np.linalg.norm(X_app - x, axis=1)
        voisins = np.argpartition(distances, n_voisins)[:n_voisins]
        labels = y_app[voisins]
        labels_unique, compteurs = np.unique(labels, return_counts=True)
        y_pred.append(labels_unique[np.argmax(compteurs)])
    return np.array(y_pred)

### Disque

In [ ]:
X_app, y_app = obtenir_disque()

In [ ]:
montrer_disque(X_app, y_app)

In [ ]:
# centre du carre / sommets du carre
X_test = np.array([[0, 0], [1, 1], [-1, 1], [1, -1], [-1, -1]])
voisins = rechercher_plus_proches_voisins(X_test, X_app)

In [ ]:
montrer_disque(X_app[voisins], y_app[voisins])

In [ ]:
# classification
y_pred = classifier_plus_proches_voisins(X_test, X_app, y_app)

In [ ]:
montrer_disque(X_test, y_pred)

In [ ]:
X_test, y_test = obtenir_disque(500)
y_pred = classifier_plus_proches_voisins(X_test, X_app, y_app)

In [ ]:
montrer_disque(X_test, y_pred)

In [ ]:
precision = np.sum(y_pred == y_test) / len(y_test)

In [ ]:
np.round(precision, 2)

### Iris

In [ ]:
X, y = obtenir_iris()

In [ ]:
X_app, X_test, y_app, y_test = partager_apprentissage_test(X, y)

In [ ]:
y_pred = classifier_plus_proches_voisins(X_test, X_app, y_app)

In [ ]:
precision = np.sum(y_pred == y_test) / len(y_test)

In [ ]:
np.round(precision, 2)

### Chiffres

In [ ]:
X, y = obtenir_chiffres()

In [ ]:
X_app, X_test, y_app, y_test = partager_apprentissage_test(X, y)

In [ ]:
# échantillons
index = obtenir_echantillons(X_test, y_test, 1)
montrer_chiffres(X_test[index], y_test[index])

In [ ]:
voisins = rechercher_plus_proches_voisins(X_test[index], X_app)

In [ ]:
index = voisins.flatten()
montrer_chiffres(X_app[index], np.repeat(np.arange(10), voisins.shape[1]))

In [ ]:
# classification
y_pred = classifier_plus_proches_voisins(X_test, X_app, y_app)

In [ ]:
precision = np.sum(y_pred == y_test) / len(y_test)

In [ ]:
np.round(precision, 2)

In [ ]:
# chiffres mal classés
index = np.where(y_pred != y_test)[0]

In [ ]:
montrer_chiffres(X_test[index], y_test[index])

In [ ]:
# classe prédite
montrer_chiffres(X_test[index], y_pred[index])

## Quelques idées à explorer

Pour l'ensemble des jeux de données :
* Étudier l'impact du nombre de plus proches voisins.
* Étudier la sensibilité au bruit (en perturbant les classes du jeu d'apprentissage).
* Retourner un score de confiance (fraction des voisins de classe majoritaire).
* Introduire une pondération permettant de donner plus de poids aux voisins les plus proches.

Pour le disque :
* Montrer les points les plus difficiles à classer.
* Changer le modèle (par exemple, deux disques qui s'intersectent).

Pour Iris :
* Montrer les zones de décision sur la forme du pétale, pour une fleur de dimensions de sépale médianes.
* Tester la normalisation de chaque dimension entre 0 (min) et 1 (max).

Pour les chiffres :
* Montrer les chiffres les plus faciles à classer puis les plus difficiles à classer.
* Tester une méthode d'égalisation des images (par exemple, somme des niveaux de gris constante).